In [79]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

dataset_name = "ecommerce_dataset"

table_clientes = "Clientes"
table_items = "Items"
table_nomes = "NomePT-EN"
table_pedidos = "Pedidos"
table_vendedores = "Vendedores"
table_Pagamentos = "Pagamentos"
table_Produtos = "Produtos"


queryClientes = f"SELECT * FROM `{client.project}.{dataset_name}.{table_clientes}`"
queryItems = f"SELECT * FROM `{client.project}.{dataset_name}.{table_items}`"
queryNomes = f"SELECT * FROM `{client.project}.{dataset_name}.{table_nomes}`"
queryPedidos = f"SELECT * FROM `{client.project}.{dataset_name}.{table_pedidos}`"
queryVendedores = f"SELECT * FROM `{client.project}.{dataset_name}.{table_vendedores}`"
queryPagamentos = f"SELECT * FROM `{client.project}.{dataset_name}.{table_Pagamentos}`"
queryProdutos = f"SELECT * FROM `{client.project}.{dataset_name}.{table_Produtos}`"

df_clientes = client.query(queryClientes).to_dataframe()
df_items = client.query(queryItems).to_dataframe()
df_nomes = client.query(queryNomes).to_dataframe()
df_pedidos = client.query(queryPedidos).to_dataframe()
df_vendedores = client.query(queryVendedores).to_dataframe()
df_pagamentos = client.query(queryPagamentos).to_dataframe()
df_produtos = client.query(queryProdutos).to_dataframe()

In [76]:
df_pedidos = df_pedidos.dropna(subset=['order_delivered_carrier_date', 'order_delivered_customer_date'])


In [ ]:
df_items['shipping_limit_date'] = pd.to_datetime(df_items['shipping_limit_date'])

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,3ee6513ae7ea23bdfab5b9ab60bffcb5,1,8a3254bee785a526d548a81a9bc3c9be,96804ea39d96eb908e7c3afdb671bb9e,2018-05-04 03:55:26+00:00,0.85,18.23
1,6e864b3f0ec71031117ad4cf46b7f2a1,1,8a3254bee785a526d548a81a9bc3c9be,96804ea39d96eb908e7c3afdb671bb9e,2018-05-02 20:30:34+00:00,0.85,18.23
2,c5bdd8ef3c0ec420232e668302179113,2,8a3254bee785a526d548a81a9bc3c9be,96804ea39d96eb908e7c3afdb671bb9e,2018-05-07 02:55:22+00:00,0.85,22.30
3,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23+00:00,1.20,7.89
4,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23+00:00,1.20,7.89


In [78]:
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_approved_at'] = pd.to_datetime(df_pedidos['order_approved_at'])
df_pedidos['order_delivered_carrier_date'] = pd.to_datetime(df_pedidos['order_delivered_carrier_date'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])
df_pedidos['order_estimated_delivery_date'] = pd.to_datetime(df_pedidos['order_estimated_delivery_date'])


In [ ]:
df_pedidos['is_delivered_late'] = df_pedidos['order_delivered_customer_date'] > df_pedidos['order_estimated_delivery_date']

  order_delivered_customer_date order_estimated_delivery_date  \
0                           NaT     2017-03-01 00:00:00+00:00   
1                           NaT     2017-05-22 00:00:00+00:00   
2                           NaT     2016-09-30 00:00:00+00:00   
3                           NaT     2016-10-24 00:00:00+00:00   
4                           NaT     2016-10-25 00:00:00+00:00   

   is_delivered_late  
0              False  
1              False  
2              False  
3              False  
4              False  


In [ ]:
query = f"""
    SELECT seller_id,
           COUNT(*) AS sales,
           SUM(price) AS total_sold,
           SAFE_DIVIDE(SUM(price), COUNT(*)) AS medium_ticket
    FROM `{client.project}.{dataset_name}.{table_items}`
    GROUP BY seller_id
"""

df_vendas = client.query(query).to_dataframe()

df_vendedores = df_vendedores.merge(df_vendas, on='seller_id', how='left')

df_vendedores.fillna({'sales': 0, 'total_sold': 0, 'medium_ticket': 0}, inplace=True)


In [ ]:
query = f"""
    SELECT product_id,
           COUNT(*) AS total_sales,
           SUM(price) AS total_revenue
    FROM `{client.project}.{dataset_name}.{table_items}`
    GROUP BY product_id
"""

df_produto_vendas = client.query(query).to_dataframe()

df_produtos = df_produtos.merge(df_produto_vendas, on='product_id', how='left')

df_produtos.fillna({'total_sales': 0, 'total_revenue': 0}, inplace=True)


In [ ]:
df_produtos

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,total_sales,total_revenue
0,a0ab96e461d74537772b84950f26a257,climatizacao,41,717,1,1050,18,7,8,3,104.70
1,20ae7c024ede613f47e0d2f23f461493,telefonia_fixa,25,455,1,330,17,11,9,1,70.00
2,4d7585daba2f8b3ed7f87447908b4237,telefonia_fixa,53,897,2,300,15,8,9,1,290.00
3,ad7aebed205805125489f8a89819b24b,construcao_ferramentas_ferramentas,41,2526,2,1150,22,10,9,1,59.99
4,980ecbcc15fe174ec1e5757c4d75b1bf,agro_industria_e_comercio,48,157,1,250,17,3,10,8,197.50
...,...,...,...,...,...,...,...,...,...,...,...
32946,e7e460583f2b78939dee0f8cd9a619ed,utilidades_domesticas,59,586,6,12250,80,10,76,2,409.80
32947,b92268ad64d3947c80fddbc3a82de845,utilidades_domesticas,55,134,1,8850,39,12,78,1,77.74
32948,4047f50df2141e2ab6c747b16b6a7bf6,utilidades_domesticas,40,564,3,30000,90,30,80,1,250.00
32949,66a4ac0f9c73e92afc9cb7566977ef83,utilidades_domesticas,58,547,5,1000,18,12,82,2,79.80


In [ ]:
df_order_totals = df_items.groupby('order_id')['price'].sum().reset_index()
df_order_totals.rename(columns={'price': 'total_order_value'}, inplace=True)

df_pedidos = df_pedidos.merge(df_order_totals, on='order_id', how='left')


df_customer_stats = df_pedidos.groupby('customer_id').agg(
    total_spent=('total_order_value', 'sum'),
    average_order_value=('total_order_value', 'mean'),
    total_orders=('order_id', 'count')
).reset_index()

df_clientes = df_clientes.merge(df_customer_stats, on='customer_id', how='left')

df_clientes.fillna({'total_spent': 0, 'average_order_value': 0, 'total_orders': 0}, inplace=True)


In [ ]:
df_clientes

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,total_spent,average_order_value,total_orders
0,2201362e68992f654942dc0067c1b716,f7d7fc0a59ef4363fdce6e3aa069d498,69900,rio branco,AC,49.90,49.90,1
1,31dbc13addc753e210692eacaea065e4,5dbba6c01268a8ad43f79157bf4454a0,69900,rio branco,AC,250.00,250.00,1
2,dad907e170748a35ef4e92238b7308f3,36b1c0516f123351ffa87430416dcae5,69900,rio branco,AC,276.14,276.14,1
3,888d2ebe1af2a8c93c75dae5dfc23719,721d1092e1a6460c67e6a0e691d899a3,69900,rio branco,AC,16.90,16.90,1
4,8a0108267d9258a0ec9f74381bc9b0de,7a2dc4682890550ebe3b8befcea3d55c,69900,rio branco,AC,69.90,69.90,1
...,...,...,...,...,...,...,...,...
99436,3fe4ae2a8958da697c995d38b5412fed,b6862b461ae74dffcaa67eae0a5360f7,77950,araguatins,TO,249.00,249.00,1
99437,6d6ebb07f1f3b14fd55086a51d049be8,f48380fe44b8161edfd5d695456110f9,77960,augustinopolis,TO,107.00,107.00,1
99438,abd4f44cab431f33b0c4f242377666f4,9b5ea59cba26d0b2742910556dad49c7,77970,praia norte,TO,79.00,79.00,1
99439,7fadd7adb85b1f419b8eb907f2159527,feeca5dd91115b235c4655f3265f3d38,77993,esperantina,TO,84.44,84.44,1


In [ ]:

receita_total = df_items['price'].sum()

vendas_por_estado = df_pedidos.merge(df_clientes, on='customer_id').groupby('customer_state')['order_id'].count()

vendas_por_categoria = df_items.merge(df_produtos, on='product_id').groupby('product_category_name')['order_id'].count()

df_resumo = pd.DataFrame({
    'Receita Total': [receita_total],
    'Vendas por Estado': [vendas_por_estado.to_dict()],
    'Vendas por Categoria': [vendas_por_categoria.to_dict()]
})


   Receita Total                                  Vendas por Estado  \
0     13591643.7  {'AC': 80, 'AL': 397, 'AM': 145, 'AP': 67, 'BA...   

                                Vendas por Categoria  
0  {'agro_industria_e_comercio': 212, 'alimentos'...  


In [ ]:
dataset_ref = client.dataset("ecommerce_analise")
try:
    client.get_dataset(dataset_ref)
    print("Dataset já existe.")
except Exception:
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {dataset_ref} criado com sucesso.")


Dataset corded-shadow-453116-f7.ecommerce_analise criado com sucesso.


In [ ]:
tabela_resumo_ref = client.dataset("ecommerce_analise").table("resumo_vendas")

job = client.load_table_from_dataframe(df_resumo, tabela_resumo_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

job.result()


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Vendas por Estado, Vendas por Categoria.
  warnings.warn(


Tabela resumo_vendas carregada com sucesso!


In [ ]:
tabela_vendedores_ref = client.dataset("ecommerce_analise").table("vendedores")
job_vendedores = client.load_table_from_dataframe(df_vendedores, tabela_vendedores_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job_vendedores.result()

tabela_clientes_ref = client.dataset("ecommerce_analise").table("clientes")
job_clientes = client.load_table_from_dataframe(df_clientes, tabela_clientes_ref, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
job_clientes.result()

Todas as tabelas transformadas foram carregadas no BigQuery com sucesso!
